<a href="https://colab.research.google.com/github/sarahgruetz/AluraChallenge_DS_2/blob/main/Alura_Challenge_semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 40.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=af0b47b9f82f407a71755201d2791c2a9159e2b6902d21ef5c876a75aaf86adf
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Alura Challenge") \
    .getOrCreate()

In [3]:
import zipfile

zipfile.ZipFile('/content/drive/MyDrive/curso-spark/ChallengeAlura/semana-2.zip','r')\
        .extractall('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados')

In [14]:
dataset = spark.read.parquet('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados/dataset_transformado_parquet')

In [8]:
dataset.show(truncate=False)

+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+-----+------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas                                                                                                                          |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu|tipo |valor |
+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+

## Seleção de Features

In [15]:
dataset\
  .select('tipo_anuncio')\
  .groupBy('tipo_anuncio')\
  .count()\
  .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [16]:
dataset\
  .select('tipo_unidade')\
  .groupBy('tipo_unidade')\
  .count()\
  .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [17]:
dataset\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|66562|
+-----------+-----+



In [18]:
dataset\
  .select('tipo')\
  .groupBy('tipo')\
  .count()\
  .show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66562|
+-----+-----+



`id` -> código de identificação do anúncio. Não será útil para o modelo.

`tipo_anuncio` -> foram selecionados apenas os registros com valor Usado, portanto essa coluna não é relevante para o modelo.

`tipo_unidade` -> foram selecionados apenas os registros com valor Apartamento, portanto essa coluna não é relevante para o modelo.

`tipo_uso` -> foram selecionados apenas os registros com valor Residencial, portanto essa coluna não é relevante para o modelo.

`tipo` -> foram selecionados apenas os registros com valor Venda, portanto essa coluna não é relevante para o modelo.

In [19]:
dataset = dataset\
              .select(
                  'andar',
                  'area_total',
                  'area_util',
                  'banheiros',
                  'caracteristicas',
                  'quartos',
                  'suites',
                  'vaga',
                  'bairro',
                  'zona',
                  'condominio',
                  'iptu',
                  'valor'
              )

## Convertendo tipo das colunas

In [20]:
dataset.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [23]:
from pyspark.sql.types import FloatType

In [24]:
dataset = dataset\
              .withColumn('area_total', dataset.area_total.cast(FloatType()))\
              .withColumn('area_util', dataset.area_util.cast(FloatType()))\
              .withColumn('condominio', dataset.condominio.cast(FloatType()))\
              .withColumn('iptu', dataset.iptu.cast(FloatType()))\
              .withColumn('valor', dataset.valor.cast(FloatType()))\

In [25]:
dataset.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: float (nullable = true)
 |-- area_util: float (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: float (nullable = true)
 |-- iptu: float (nullable = true)
 |-- valor: float (nullable = true)



## Arrumando a coluna `caracteristicas`

In [26]:
from pyspark.sql import functions as f

In [32]:
dataset.withColumn('caracteristicas',
                   f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col("caracteristicas")))\
                   .show()

+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|  iptu|   valor|
+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+------+--------+
|    0|      43.0|     43.0|        1|[Churrasqueira, A...|      2|  null|   1|            Realengo|  Zona Oeste|     285.0|  null| 22999.0|
|    0|      44.0|     44.0|        1|                null|      2|     0|   0|               Irajá|  Zona Norte|     170.0|   0.0|110000.0|
|    4|      null|     55.0|        1|                null|      2|     0|   1|              Cosmos|  Zona Oeste|      null|  null|115000.0|
|    2|      null|     55.0|        1|                null|      2|     0|   0|        Tomás Coelho|  Zona Norte|      null|  null|115000.0|
|    0|      

In [33]:
dataset = dataset.withColumn('caracteristicas',
                   f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col("caracteristicas")))

## Lidando com os valores nulos